In [ ]:
# default_exp helpers

# helpers
> this didn't fit anywhere else

In [ ]:
#export
import numpy as np
import matplotlib.pyplot as plt

def plot_regret(y_optimal_list,y_hat_list):
    y_optimal_array = np.array(y_optimal_list)
    y_hat_array = np.array(y_hat_list)
    regret_list = []


    regret = np.cumsum(y_optimal_array - y_hat_array)

    plt.plot(regret)